## Import

In [1]:
import pandas as pd
import numpy as np
import time
# BERT
from transformers import BertModel, BertTokenizer
# RoBERTa
from transformers import RobertaModel, RobertaTokenizer
# DistilBERT
# from transformers import DistilBertModel, DistilBertTokenizer
# XLMRoBERTa
# pip install sentencepiece
from transformers import XLMRobertaTokenizer, XLMRobertaModel
# 유사도
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import warnings;warnings.filterwarnings('ignore')

C:\Users\ha\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Read data

In [2]:
BRAND_raw = pd.read_csv('PATH', encoding='utf-8')[['VARIANT_NM']].drop_duplicates()
BRAND_product = pd.read_csv('PATH', encoding='cp949')[['ORIGINAL_NM']].drop_duplicates()
print(f'{BRAND_raw.shape[0]}개 변형 상품명을 {BRAND_product.shape[0]}개 기준 상품명으로 변경합니다.')

## Generate TrainDataSet

In [ ]:
# ORIGINAL_NM을 DB에 적재할 값(DC_ORIGINAL_NM)으로 가공한다.(행사나 주문일 제거, 특수기호 수정 등)
# VARIANT_NM을 DC_ORIGINAL_NM(Target, 정답)을 붙이게 DC_VARIANT_NM으로 가공 후 병합한다..

## 상품명 변환 AI
SATUR, RAWROW로 실험해보며 정한 규칙을 적는다.
- 여러 언어 상품명이 있어도 RAW한 상품명(VARIANT_NM)과 도출해야 하는 실제 상품명(DC_ORIGINAL_NM)으로 input되게 `concat`하여 사용한다.
- 한 상품명에 여러 언어가 섞인 경우에도 VARIANT_NM, DC_ORIGINAL_NM으로만 예측한다.
- DistilBERT를 제외하고 상품명 인식에 뛰어난 성능을 보인 `BERT`, `roBERTa`와 다국어 지원 모델인 `XLM`, `BERT-MULTI`로 Embedding을 한다.

In [108]:
# 상품명에 언어가 혼재되지 않은 경우 사용할 함수
def singleEncode(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state[:, 0, :].numpy()

def singleEmbedding(names):
    embeddings = [singleEncode(N) for N in names]
    return embeddings

In [108]:
# 상품명에 언어가 혼재된 경우 사용할 함수
def multiEncode(text):
    # 평균을 반환하여 상품 정보가 고르게 반영되도록 조정함
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1)

def multiEmbedding(names):
    embeddings = [multiEncode(N) for N in names]
    return embeddings

In [105]:
# 코사인 유사도 반환 함수
def Similarity(original_embeddings, variant_embeddings):
    cs = []
    for VE in variant_embeddings:
        cs.append([cosine_similarity(VE, OE)[0][0] for OE in original_embeddings])
    return cs

### 0. 임시 코드
BERT, DistilBERT token load를 임시로 적어둡니다.
```python
# 1. BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 2. DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
```

## 2. roBERTa
- `모델 설명`

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# 원본(자사몰 내지 고객의 상품 테이블), 변형(판매처 별 상품명)
original_embeddings, variant_embeddings = singleEmbedding(BRAND['DC_Multi_nm']), singleEmbedding(BRAND['pro_nm'])

# 변형된 상품명 별 원본 상품명과의 유사도
roberta = pd.DataFrame(Similarity(original_embeddings, variant_embeddings))

In [ ]:
BRAND['predict'] = BRAND['DC_Multi_nm'].iloc[roberta.apply(lambda x: x.argmax()).values].values
print('병합 시 상품명 정확도:', sum(BRAND['DC_Multi_nm']==BRAND['predict'])/BRAND.shape[0])

## 3. XLM-roBERTa 
- `모델 설명`

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

# 원본(자사몰 내지 고객의 상품 테이블), 변형(판매처 별 상품명)
original_embeddings, variant_embeddings = multiEmbedding(BRAND['DC_Multi_nm']), multiEmbedding(BRAND['pro_nm'])

# 변형된 상품명 별 원본 상품명과의 유사도
xlm = pd.DataFrame(Similarity(original_embeddings, variant_embeddings))

In [15]:
BRAND['predict'] = BRAND['DC_Multi_nm'].iloc[xlm.apply(lambda x: x.argmax()).values].values
print('병합 시 상품명 정확도:', sum(BRAND['DC_Multi_nm']==BRAND['predict'])/BRAND.shape[0])

병합 시 상품명 정확도: 0.913075780089153


## 4. BERT-Multilingual
- `모델 설명`

In [109]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased')

# 원본(자사몰 내지 고객의 상품 테이블), 변형(판매처 별 상품명)
original_embeddings, variant_embeddings = multiEmbedding(BRAND['DC_Multi_nm']), multiEmbedding(BRAND['pro_nm'])

# 변형된 상품명 별 원본 상품명과의 유사도
bert_multi = pd.DataFrame(Similarity(original_embeddings, variant_embeddings))

In [67]:
BRAND['predict'] = BRAND['DC_Multi_nm'].iloc[bert_multi.apply(lambda x: x.argmax()).values].values
print('병합 시 상품명 정확도:', sum(BRAND['DC_Multi_nm']==BRAND['predict'])/BRAND.shape[0])

병합 시 상품명 정확도: 0.9702823179791976


## Ensemble
- 모델의 정확도, 시간, 다중공선성(0.2 이하)을 고려하여 Ensemble한다.
  ```python
    # Satur 모델링 결과를 예시로 적어둡니다.
    score = pd.DataFrame({'model':['roBERTa','XLM','BERT_multi'],
                          'precision':[0.9777117384843982, 0.913075780089153, 0.9702823179791976],
                          'time':[7.2, 9.7, 9.4]})

    result = [roberta, xlm, bert_multi]
    corr = []
    for i in range(3):
        for j in range(i+1,3):
            corr.append([score['model'].loc[i], score['model'].loc[j], np.corrcoef(result[i], result[j]).mean()])

    # 다중공선성 시각화
    import seaborn as sns
    sns.heatmap(pd.pivot_table(pd.DataFrame(corr), index=0, columns=1, values=2), annot=True, cmap='Blues')
    ```
- 1. 세 모델의 유사도를 산술평균 후 특정값일 확률을 구해 threshold 미만이면 결측을 반환하게 한다.<br>
  2. 세 모델 의사결정을 다수결하여 특정값을 도출하지 못하면 결측을 반환하게 한다.<br>
  3. 세 모델의 유사도 산술평균값을 input하여 상품명을 예측하는 stacking모델을 구현한다.

In [ ]:
# transpos됨 돌려야함!

In [130]:
bert, roberta, distilbert, xlm, bert_multi, _, _, _, _ = pickle.load(open('../data/satur/submission/240327_satur_modeling.pkl', 'rb'))

In [132]:
# satur['predict'] = satur['DC_Multi_nm'].iloc[bert.T.apply(lambda x: x.argmax()).values].values
sum(satur['predict']==satur['DC_Multi_nm'])/satur.shape[0]

0.9806835066864784

In [133]:
satur['predict'] = satur['DC_Multi_nm'].iloc[xlm.T.apply(lambda x: x.argmax()).values].values
sum(satur['predict']==satur['DC_Multi_nm'])/satur.shape[0]

0.9814264487369985

In [140]:
satur['predict'] = satur['DC_Multi_nm'].iloc[mean.T.apply(lambda x: x.argmax()).values].values
sum(satur['predict']==satur['DC_Multi_nm'])/satur.shape[0]

0.9829123328380386

In [142]:
satur['predict'] = satur['DC_Multi_nm'].iloc[mean.T.apply(lambda x: x.argmax()).values].values
sum(satur['predict']==satur['DC_Multi_nm'])/satur.shape[0]

0.9829123328380386

In [162]:
mean = pd.DataFrame((bert.values+xlm.values+bert_multi.values)/3, index=roberta.index,columns=roberta.columns)

In [8]:
satur_raw = pd.read_csv('../data/satur/satur_20240313.csv',
                        names=['nft_req_num', 'pro_nm', 'pro_cd', 'user_idx', 'price', 'ref_order_id', 'order_dt', 'store_nm'],
                        encoding='utf-8')[['pro_nm']]
satur_product = pd.read_csv('../data/satur/satur_product.csv', usecols=['enc_nm','kor_nm'], encoding='cp949')

## Train DataSet

### $\blacktriangleright$ Satur

In [9]:
satur = satur_raw.drop_duplicates().reset_index(drop=True).reset_index()

# 불필요한 특수문자를 제거한다.
# [참고] MassAdoption\3월\240313_SaturAssociationRule\1. DataCleansing\3. Cleansing_ProductTable.ipynb
satur['DC_pro_nm'] = satur.pro_nm.apply(lambda x: x[x.rindex('>')+1:] if '<' in x else x)\
                    .apply(lambda x: x[x.rindex(')')+1:] if '(' in x else x)\
                    .apply(lambda x: x[x.rindex(']')+1:] if '[' in x else x)
satur.shape

(1469, 3)

In [10]:
satur_product.fillna('-', inplace=True)
satur_product = satur_product.drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'product_idx'})

# 불필요한 특수문자를 제거한다.
# [참고] MassAdoption\3월\240313_SaturAssociationRule\1. DataCleansing\3. Cleansing_ProductTable.ipynb
satur_product['DC_enc_nm'] = satur_product['enc_nm'].apply(lambda x: x[x.rindex('>')+1:] if '<' in x else x)\
                            .apply(lambda x: x[x.rindex(')')+1:] if '(' in x else x)\
                            .apply(lambda x: x[x.rindex(']')+1:] if '[' in x else x).str.replace('\n','').str.strip()
satur_product['DC_kor_nm'] = satur_product['kor_nm'].apply(lambda x: x[x.rindex('>')+1:] if '<' in x else x)\
                            .apply(lambda x: x[x.rindex(')')+1:] if '(' in x else x)\
                            .apply(lambda x: x[x.rindex(']')+1:] if '[' in x else x).str.replace('\n','').str.strip()
satur_product.shape

(965, 5)

In [11]:
enc = satur.merge(satur_product, left_on='DC_pro_nm', right_on='DC_enc_nm')
kor = satur.merge(satur_product, left_on='DC_pro_nm', right_on='DC_kor_nm')
print('병합되지 않은 데이터 비율:', (1-((enc.shape[0]+kor.shape[0])/1469))*100)

병합되지 않은 데이터 비율: 8.373042886317227


In [12]:
satur = pd.concat([enc[['pro_nm','DC_enc_nm']].rename(columns={'DC_enc_nm':'DC_Multi_nm'}),
                   kor[['pro_nm','DC_kor_nm']].rename(columns={'DC_kor_nm':'DC_Multi_nm'})])

In [151]:
# 1. 임계값 조정
ease = mean.apply(lambda x: np.clip(x, 0, 1)).apply(lambda x: [x.max(), x.argmax()]).T

ease = ease.apply(lambda x: np.nan if x[0] < 0.95 else x[1], axis=1)

under_threshold = ease[ease.isna()].index

over_threshold = satur.iloc[ease.dropna()]

In [152]:
print('병합 시 상품명 정확도:', sum(over_threshold['DC_Multi_nm']==over_threshold['predict'])/over_threshold.shape[0])

병합 시 상품명 정확도: 1.0


In [153]:
over_threshold.shape

(1207, 3)

In [ ]:
# index가 잘못 붙음

In [164]:
satur[['DC_Multi_nm']].merge(satur_product[['product_idx','DC_enc_nm']], left_on='DC_Multi_nm', right_on='DC_enc_nm')

,DC_Multi_nm,product_idx,DC_enc_nm
0,Cafri Citron Drawing Graphic Sweatshirts - Cla...,307,Cafri Citron Drawing Graphic Sweatshirts - Cla...
1,Endless Saturday Hoodie - Classic Black,315,Endless Saturday Hoodie - Classic Black
2,Classic Letter Logo Hoodie - Classic Black,312,Classic Letter Logo Hoodie - Classic Black
3,Classic Dyed Cable Knit - Night Black,392,Classic Dyed Cable Knit - Night Black
4,Classic Dyed Cable Knit - Night Black,841,Classic Dyed Cable Knit - Night Black
...,...,...,...
1029,Dublin Unbalanced Chain Embroidery Sweatshirts...,329,Dublin Unbalanced Chain Embroidery Sweatshirts...
1030,Vinales Retro 2-Tuck Denim Pants - Light Blue,481,Vinales Retro 2-Tuck Denim Pants - Light Blue
1031,Leeds Wool Single Blazer - Vanta Black,54,Leeds Wool Single Blazer - Vanta Black
1032,Cafri Citron Heavy Cotton Sweatshirts - Melang...,301,Cafri Citron Heavy Cotton Sweatshirts - Melang...


In [167]:
satur[['DC_Multi_nm']].merge(satur_product[['product_idx','DC_kor_nm']], left_on='DC_Multi_nm', right_on='DC_kor_nm')

,DC_Multi_nm,product_idx,DC_kor_nm
0,구아타 자르댕 : 태양의 사물 볼캡 리조트 아이보리,648,구아타 자르댕 : 태양의 사물 볼캡 리조트 아이보리
1,테오 코튼 올 데이 후드 집업 팔레트 블랙,160,테오 코튼 올 데이 후드 집업 팔레트 블랙
2,테오 코튼 올 데이 후드 집업 팔레트 블랙,767,테오 코튼 올 데이 후드 집업 팔레트 블랙
3,테오 코튼 올 데이 후드 집업 팔레트 블랙,160,테오 코튼 올 데이 후드 집업 팔레트 블랙
4,테오 코튼 올 데이 후드 집업 팔레트 블랙,767,테오 코튼 올 데이 후드 집업 팔레트 블랙
...,...,...,...
715,클래식 첼시 데님 팬츠 미디움 워시드 블루,482,클래식 첼시 데님 팬츠 미디움 워시드 블루
716,클래식 스몰 로고 리벳 후드 클래식 네이비,297,클래식 스몰 로고 리벳 후드 클래식 네이비
717,클래식 스몰 로고 리벳 후드 클래식 네이비,825,클래식 스몰 로고 리벳 후드 클래식 네이비
718,클래식 스몰 로고 리벳 후드 클래식 네이비,297,클래식 스몰 로고 리벳 후드 클래식 네이비


In [155]:
1207/1346

0.8967310549777118

In [123]:
satur

,pro_nm,DC_Multi_nm,predict
0,Cafri Citron Drawing Graphic Sweatshirts - Cla...,Cafri Citron Drawing Graphic Sweatshirts - Cla...,Cafri Citron Drawing Graphic Sweatshirts - Cla...
1,Endless Saturday Hoodie - Classic Black,Endless Saturday Hoodie - Classic Black,Endless Saturday Hoodie - Classic Black
2,Classic Letter Logo Hoodie - Classic Black,Classic Letter Logo Hoodie - Classic Black,Classic Letter Logo Hoodie - Classic Black
3,Classic Dyed Cable Knit - Night Black,Classic Dyed Cable Knit - Night Black,Classic Dyed Cable Knit - Night Black
4,Classic Dyed Cable Knit - Night Black,Classic Dyed Cable Knit - Night Black,Classic Dyed Cable Knit - Night Black
...,...,...,...
526,심볼 브로치 데님 트러커 자켓 라이트 워시드 블루,심볼 브로치 데님 트러커 자켓 라이트 워시드 블루,Faro Cashmere Blend Collar Knit Cardigan - Cla...
527,이비자 네츄럴 메시드 니트 퍼시몬 핑크,이비자 네츄럴 메시드 니트 퍼시몬 핑크,Weekend Sunburst Swim Shorts
528,클래식 첼시 데님 팬츠 미디움 워시드 블루,클래식 첼시 데님 팬츠 미디움 워시드 블루,Keller Highwaist Carpenter Pants - Midnight Navy
529,클래식 스몰 로고 리벳 후드 클래식 네이비,클래식 스몰 로고 리벳 후드 클래식 네이비,Satur Keyring Leather Airpods Case - Pure Ivory


## Save Data